# Applied Data Science Capstone


### Introduction/Business Problem



#### As a student living in Williamsburg, VA, I feel that restaurants are not enough, especially Chinese restaurants. People in Williamsburg deserve to try Chinese famous type of food - Hot Pot. Therefore, this capstone is going to find the best location to open a restaurants according to the location's reviews, popularity and other factors. The data will be scraped from Foursquare and the problem will be solved using Foursquare API.

### Data 

#### In this step, I will request data from foursquare and use the data to build a datafame containing information that tells me detail about restaurants in the location. By doing this, I can further find a location that's the best for opening a new restaurant. The coordinate of center is chosen as Governor's Palace since this is one of the most famous location in Williamsburg, everyone will go there and everyone lives near the place. In this part, we mainly want to know what the data look like, instead of using data to do the analysis

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
#!pip install shapely
#import shapely.geometry
#!pip install pyproj
import pyproj
import math
import pickle

#### Get client id and client secret from foursquare developer

In [2]:
CLIENT_ID = 'IIM3ZPX4O0MTECO31PIVRFK0UQMYYAJNVRWABOAJLCYEFRF0' # your Foursquare ID
CLIENT_SECRET = 'KVV1LN5ITDZ4YDV3FGXM2WLSOL5OVSPMR3GQPYFNIOFACQBO' # your Foursquare Secret
VERSION = '20200401'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IIM3ZPX4O0MTECO31PIVRFK0UQMYYAJNVRWABOAJLCYEFRF0
CLIENT_SECRET:KVV1LN5ITDZ4YDV3FGXM2WLSOL5OVSPMR3GQPYFNIOFACQBO


In [3]:
address = "Governor's Palace,Williamsburg, VA"
'''
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print(latitude, longitude)
'''

def get_coordinates(address, verbose=False):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    lat = location.latitude
    lon = location.longitude
    return [lat,lon]

williamsburg_center = get_coordinates(address)
print('Coordinate of {}: {}'.format(address, williamsburg_center))
latitude = williamsburg_center[0]
longitude = williamsburg_center[1]

Coordinate of Governor's Palace,Williamsburg, VA: [37.2742497, -76.7021709]


In [8]:
search_query = 'Restaurant'
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [9]:
# Define the corresponding URL
#url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

#### It's not necessary to print results unless you want to see what's actually in it, because it'll be used many time in analysis part

In [10]:
# Send the GET Request and examine the results
results = requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '5e8a8237da9e14001b1949f8'}, 'response': {'venues': [{'id': '4b5b270cf964a520e3e628e3', 'name': 'The Trellis Restaurant', 'location': {'address': '403 W Duke of Gloucester St', 'crossStreet': 'N Henry St', 'lat': 37.27108130431542, 'lng': -76.70615589922397, 'labeledLatLngs': [{'label': 'display', 'lat': 37.27108130431542, 'lng': -76.70615589922397}], 'distance': 499, 'postalCode': '23185', 'cc': 'US', 'city': 'Williamsburg', 'state': 'VA', 'country': 'United States', 'formattedAddress': ['403 W Duke of Gloucester St (N Henry St)', 'Williamsburg, VA 23185', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d14e941735', 'name': 'American Restaurant', 'pluralName': 'American Restaurants', 'shortName': 'American', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1586135623', 'hasPerk': False}, {'id': '4bb0e18ff964a5203c663ce3', 'name': "Berret's Seafood Restaurant and

#### Create a dataframe contains every information that may be needed

In [11]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4b5b270cf964a520e3e628e3,The Trellis Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1586135623,False,403 W Duke of Gloucester St,N Henry St,37.271081,-76.706156,"[{'label': 'display', 'lat': 37.27108130431542...",499,23185,US,Williamsburg,VA,United States,"[403 W Duke of Gloucester St (N Henry St), Wil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bb0e18ff964a5203c663ce3,Berret's Seafood Restaurant and Taphouse Grill,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1586135623,False,199 S Boundary St,Merchant's Square,37.270027,-76.707125,"[{'label': 'display', 'lat': 37.27002662899438...",643,23185,US,Williamsburg,VA,United States,"[199 S Boundary St (Merchant's Square), Willia...",2044294,https://www.grubhub.com/restaurant/berrets-sea...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,36256198
2,4c73ec6aec8d952110def1c2,Peking Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1586135623,False,120 Waller Mill Rd Ste J,Bypass Rd,37.288596,-76.717956,"[{'label': 'display', 'lat': 37.28859584124185...",2122,23185,US,Williamsburg,VA,United States,"[120 Waller Mill Rd Ste J (Bypass Rd), William...",NaN,NaN,NaN,NaN,NaN,NaN,445345761
3,4daa52174df01c19b1886570,The Veranda Restaurant (Fort Magruder Hotel),"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1586135623,False,NaN,NaN,37.263629,-76.679951,"[{'label': 'display', 'lat': 37.26362902781627...",2296,NaN,US,Williamsburg,VA,United States,"[Williamsburg, VA, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c3243067cc0c9b63005f09a,The Jefferson Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1586135623,False,1453 Richmond Rd,NaN,37.289970,-76.722190,"[{'label': 'display', 'lat': 37.28997, 'lng': ...",2491,23185,US,Williamsburg,VA,United States,"[1453 Richmond Rd, Williamsburg, VA 23185, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
food_category = '4bf58dd8d48988d14e941735'
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,The Trellis Restaurant,American Restaurant,403 W Duke of Gloucester St,N Henry St,37.271081,-76.706156,"[{'label': 'display', 'lat': 37.27108130431542...",499,23185,US,Williamsburg,VA,United States,"[403 W Duke of Gloucester St (N Henry St), Wil...",4b5b270cf964a520e3e628e3
1,Berret's Seafood Restaurant and Taphouse Grill,Seafood Restaurant,199 S Boundary St,Merchant's Square,37.270027,-76.707125,"[{'label': 'display', 'lat': 37.27002662899438...",643,23185,US,Williamsburg,VA,United States,"[199 S Boundary St (Merchant's Square), Willia...",4bb0e18ff964a5203c663ce3
2,Peking Restaurant,Japanese Restaurant,120 Waller Mill Rd Ste J,Bypass Rd,37.288596,-76.717956,"[{'label': 'display', 'lat': 37.28859584124185...",2122,23185,US,Williamsburg,VA,United States,"[120 Waller Mill Rd Ste J (Bypass Rd), William...",4c73ec6aec8d952110def1c2
3,The Veranda Restaurant (Fort Magruder Hotel),American Restaurant,NaN,NaN,37.263629,-76.679951,"[{'label': 'display', 'lat': 37.26362902781627...",2296,NaN,US,Williamsburg,VA,United States,"[Williamsburg, VA, United States]",4daa52174df01c19b1886570
4,The Jefferson Restaurant,American Restaurant,1453 Richmond Rd,NaN,37.289970,-76.722190,"[{'label': 'display', 'lat': 37.28997, 'lng': ...",2491,23185,US,Williamsburg,VA,United States,"[1453 Richmond Rd, Williamsburg, VA 23185, Uni...",4c3243067cc0c9b63005f09a


#### Now we have a basic idea of restaurants in Williamsburg and we have the coordinate of the center of Williamsburg. Also, we know the distances between each retaurants and the center of Williamsburg

#### Now lets take a look of rating for The Trellis Restaurant

In [14]:
venue_id = '4b5b270cf964a520e3e628e3' # ID of The Trellis Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


## Analysis



#### Since we can get coordinates from address, lets try getting address from coordinates. In this part, we are going to find neighborhoods around the center of Williamsburg, also, we will find the number of restaurants in the neighborhoods. The final locations for our restaurant is going to be around the center of Williamsburg. 

#### First, we need to know the address of the center of Williamsburg. The function will be used later to access not only for address of center Williamsburg, but also for other locations. Here we are able to define latitude and longitude because we got the coordinate from before.

In [15]:
def get_address(latitude, longitude, verbose=False):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    response = requests.get(url).json()
    #print(response)
    if verbose:
        print('Foursquare Maps API JSON result =>', response)
    #print(response)
    results = response['response']['venues']
    #print(results)
    address = results[0]['location']['formattedAddress']
    #print(address)
    #return address
    return(', '.join(address))

    
add = get_address(latitude, longitude)
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(latitude, longitude, add))

Reverse geocoding check
-----------------------
Address of [37.2742497, -76.7021709] is: 403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States


#### We need to know both the coordinate of a location and the *** distance in meters *** for later calculation. WGS84 is spherical coordinate system.

In [16]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transforming')
print('-------------------------------')
print("Governor's Palace longitude={}, latitude={}".format(longitude, latitude))
x, y = lonlat_to_xy(longitude, latitude)
print("Governor's Palace UTM X={}, Y={}".format(x, y))
lo, la = xy_to_lonlat(x, y)
print("Governor's Palace longitude={}, latitude={}".format(lo, la))

Coordinate transforming
-------------------------------
Governor's Palace longitude=-76.7021709, latitude=37.2742497
Governor's Palace UTM X=-6422849.218666615, Y=10246073.437448127
Governor's Palace longitude=-76.70217090000224, latitude=37.27424969999654


In [17]:
center_x, center_y = lonlat_to_xy(longitude, latitude)

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = center_x - 6000
x_step = 600
y_min = center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(center_x, center_y, x, y)
        if (distance_from_center <= 2001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
print(len(latitudes), 'candidate neighborhood centers generated.')

42 candidate neighborhood centers generated.


In [18]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    #address = address.remove('United States') # We don't need country part of address
    addresses.append(address)
    print('.', end='')
print(' done.')

Obtaining location addresses: .......................................... done.


In [19]:
addresses[25:35]

["199 S Boundary St (Merchant's Square), Williamsburg, VA 23185, United States",
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States',
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States',
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States',
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States',
 '120 Waller Mill Rd Ste J (Bypass Rd), Williamsburg, VA 23185, United States',
 "199 S Boundary St (Merchant's Square), Williamsburg, VA 23185, United States",
 "199 S Boundary St (Merchant's Square), Williamsburg, VA 23185, United States",
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States',
 '403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States']

#### Create a new dataframe for locations, notice that although the address are the same, the coordinates are different. This is probably caused by the city is small

In [20]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(5)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"403 W Duke of Gloucester St (N Henry St), Will...",37.268093,-76.691090,-6.424049e+06,1.024451e+07,1967.231557
1,"403 W Duke of Gloucester St (N Henry St), Will...",37.271378,-76.691289,-6.423449e+06,1.024451e+07,1670.329309
2,"403 W Duke of Gloucester St (N Henry St), Will...",37.274664,-76.691488,-6.422849e+06,1.024451e+07,1558.845727
3,"403 W Duke of Gloucester St (N Henry St), Will...",37.277950,-76.691687,-6.422249e+06,1.024451e+07,1670.329309
4,"403 W Duke of Gloucester St (N Henry St), Will...",37.281237,-76.691886,-6.421649e+06,1.024451e+07,1967.231557


In [21]:
def get_categories(categories):
    return [(item['name'], item['id']) for item in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address


def get_venues_near_location(lat, lon, category, client_id, client_secret, radius, limit):
    version = '20200401'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    results = requests.get(url).json()['response']['groups'][0]['items']
    #print(requests.get(url).json()['response'])
    #print(results)
    venuess = [(item['venue']['id'],
               item['venue']['name'],
               get_categories(item['venue']['categories']),
               (item['venue']['location']['lat'], item['venue']['location']['lng']),
               format_address(item['venue']['location']),
               item['venue']['location']['distance']) for item in results]   
    return venuess


#### Create a function to check grab and load restaurants in the area

In [22]:
def is_restaurant(categories):
    restaurant_words = ['restaurant','food','steakhouse','pizza']
    restaurant = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
    return restaurant


def get_restaurants(lats, lons):
    restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius, LIMIT)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res = is_restaurant(venue_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
location_restaurants = []
loaded = False
'''
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass
'''
# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [23]:
print('Total number of restaurants:', len(restaurants))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 53
Average number of restaurants in neighborhood: 0.8095238095238095


In [24]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4b22a52ef964a520f94a24e3', 'Second Street, An American Bistro', 37.27457674108086, -76.68710623649967, '140 Second St (Page St), Williamsburg, VA 23185, United States', 2405, -6422896.003620776, 10243877.507414116)
('4b351c2bf964a5207e2a25e3', "King's Arms Tavern", 37.27143591439646, -76.69561146881775, '409 E Duke of Gloucester St, Williamsburg, VA 23185, United States', 1854, -6423408.2357190065, 10245143.42603718)
('4bb4d752bc82a5938dec3e72', "Chowning's Tavern", 37.27139563256916, -76.69934524195013, '109 E Duke of Gloucester St, Williamsburg, VA 23185, United States', 1588, -6423389.164093698, 10245687.381857341)
('4b450d2ff964a5208b0226e3', 'Blue Talon Bistro', 37.27208190581959, -76.70696101568169, '420 Prince George St (N Henry St), Williamsburg, VA 23185, United States', 1081, -6423210.097254905, 10246790.039706493)
('4bb61cdd6edc76b02068301c', "Shield's Tavern", 37.27134212143677, -76.69480830430984, '422 E Duke of Gloucester 

#### Now add the number of restaurants in the neibourhoods. Here our radius is 5000m since the city is a small city. If the radius is too small, there will be too many 0s in the list of "restaurants in area"

In [40]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=5000m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=5000m: 0.8095238095238095


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"403 W Duke of Gloucester St (N Henry St), Will...",37.268093,-76.691090,-6.424049e+06,1.024451e+07,1967.231557,0
1,"403 W Duke of Gloucester St (N Henry St), Will...",37.271378,-76.691289,-6.423449e+06,1.024451e+07,1670.329309,1
2,"403 W Duke of Gloucester St (N Henry St), Will...",37.274664,-76.691488,-6.422849e+06,1.024451e+07,1558.845727,0
3,"403 W Duke of Gloucester St (N Henry St), Will...",37.277950,-76.691687,-6.422249e+06,1.024451e+07,1670.329309,1
4,"403 W Duke of Gloucester St (N Henry St), Will...",37.281237,-76.691886,-6.421649e+06,1.024451e+07,1967.231557,1
5,"403 W Duke of Gloucester St (N Henry St), Will...",37.266312,-76.694550,-6.424349e+06,1.024503e+07,1824.828759,0
6,"403 W Duke of Gloucester St (N Henry St), Will...",37.269598,-76.694750,-6.423749e+06,1.024503e+07,1374.772708,5
7,"403 W Duke of Gloucester St (N Henry St), Will...",37.272883,-76.694949,-6.423149e+06,1.024503e+07,1081.665383,2
8,"403 W Duke of Gloucester St (N Henry St), Will...",37.276169,-76.695149,-6.422549e+06,1.024503e+07,1081.665383,0
9,"403 W Duke of Gloucester St (N Henry St), Will...",37.279456,-76.695348,-6.421949e+06,1.024503e+07,1374.772708,0


#### With the neighborhoods  that have more than 0 restaurants (meaning there are people come and go around the area) and fewer than 2 (we don't want too many competitors  since we are new here).

In [27]:
good_res_count1 = np.array((df_locations['Restaurants in area'] <= 2))
print('Locations with no more than two restaurants nearby:', good_res_count1.sum())

good_res_count2 = np.array((df_locations['Restaurants in area'] > 0))
print('Locations with no more than two restaurants nearby:', good_res_count2.sum())

good_locations = np.logical_and(good_res_count1, good_res_count2)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]

Locations with no more than two restaurants nearby: 38
Locations with no more than two restaurants nearby: 14
Locations with both conditions met: 10


In [28]:
number_of_clusters = 5

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

In [38]:
candidate_area_addresses = []
print('--------------------------------------------------------------')
print('Best place recommended for opening a Chinese Hotpot Restaurant')
print('--------------------------------------------------------------\n')
for lon, lat in cluster_centers:
    addr = get_address(lat, lon)
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, center_x, center_y)
print("{}{} => {:.1f}km from Governor's Palace".format(addr, ' '*(50-len(addr)), d/1000))

--------------------------------------------------------------
Best place recommended for opening a Chinese Hotpot Restaurant
--------------------------------------------------------------

403 W Duke of Gloucester St (N Henry St), Williamsburg, VA 23185, United States => 1.3km from Governor's Palace


In [42]:
map_ = folium.Map(location=williamsburg_center, zoom_start=14)
folium.Circle(williamsburg_center, radius=25, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_) 
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_)
map_